In [4]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.sparse import diags
from scipy.linalg import block_diag
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from functools import singledispatch

In [100]:
class Bspline():
    
    def basisfunction(self, X, knots, j, l):
        """B-spline basis function definition according to Fahrmeir, Regression p.429.

        Parameters:
        -----------
        X :      array   - Input data of shape (n_samples, ) to evaluate the B-spline basis function 
                           of order l on.
        knots :  array   - Knot sequence defining the B-spline basis function.
        j :      int     - Index of the B-spline basis function to evaluate.
        l :      int     - Order of the B-spline basis function, e.g. l=3 -> cubic.

        Returns:
        --------
        b : array     - B-spline basis function evaluated at X. 

        """
        if l == 0:
            b = ((knots[j] <= X) & (X < knots[j+1])).astype(int)
            return b
        else:
            b0 = (X - knots[j-l]) / (knots[j] - knots[j-l])
            b1 = (knots[j+1] - X) / (knots[j+1] - knots[j+1-l])
            b = b0*self.basisfunction(X, knots, j-1, l-1) + b1*self.basisfunction(X, knots, j, l-1)
            return b


    def basismatrix(self, X, nr_splines=10, l=3, knot_type="e"):
        """Generate the B-spline basis matrix for nr_splines given the data X.

         Note: (nr_splines + l + 1) knots are needed for a B-spline basis of 
               order l with nr_splines, e.g. for l=3, nr_splines=10 -> len(knots) = 14. 

         Parameters:
         ----------
         X :          array  -  Input data of shape (n_samples, ) to compute the B-spline basis matrix for.
         nr_splines : int    -  Number of parameters (== number of B-spline basis functions).
         l :          int    -  Specifies the order of the B-spline basis functions.
         knot_type :  str    -  Decide between equidistant "e" and quantile based "q"
                                knot placement.

         Returns:
         --------
         B : matrix  - B-spline basis matrix of shape: ( length(x) x nr_splines ).
         k : array   - Knot sequence.

        """
        assert (len(X.shape) == 1), "Argument X is of wrong shape."
        B = np.zeros((len(X), nr_splines))
        xmin, xmax = X.min(), X.max()

        if knot_type is "e":
            knots_inner = np.linspace(xmin, xmax, nr_splines-l+1)
        elif knot_type is "q":
            p = np.linspace(0, 1, nr_splines-l+1);
            xs = np.sort(X, kind="quicksort")
            quantile_idx = np.array((len(X)-1)*p, dtype=np.int16)
            knots_inner = xs[quantile_idx]
        else:
            print(f"Knot Type {knot_type} not implemented!")
        
        dknots = np.diff(knots_inner).mean()
        knots_left = np.linspace(xmin-l*dknots, xmin-dknots, l)
        knots_right = np.linspace(xmax+dknots, xmax+l*dknots, l)
        knots = np.concatenate((knots_left, knots_inner, knots_right))

        for j in range(l,len(knots)-1):
            B[:,j-l] = self.basisfunction(X, knots, j, l);

        self.knot_type = knot_type
        self.l = l
        self.basis = B
        self.knots = knots
        return dict(basis=B, knots=knots)

    def fit(self, X, y, nr_splines=10, l=3, knot_type="e"):
        """Calculate the least squares parameters of the B-spline given the data X.

        Parameters:
        -----------
        X : array        - Input data of shape (n_samples, ).
        y : array        - Output data of shape (n_samples, ).
        nr_splines : int - Nr. of B-spline basis functions to use.
        l : int          - Order of the B-spline basis functions.
        knot_type : str  - Specifies the type of the knot sequence, either "e" or "q".

        Returns:
        --------
        coef_ : array  - Estimated coefficients of shape (nr_splines, ).
        B : matrix     - B-spline basis matrix.
        k : array      - Knot sequence 
        """
        if len(X.shape) == 1:
            B, k = self.basismatrix(X=X, nr_splines=nr_splines, l=l, knot_type=knot_type).values()
        elif X.shape[1] == 2:
            B, k1, k2 = self.tensorproduct_basismatrix(X=X, nr_splines=nr_splines, l=l, knot_type=knot_type).values()
            k = (k1, k2)
        else:
            print("Maximal dimension == 2!")
            return 
        solution = np.linalg.lstsq(a=B, b=y, rcond=None)      
        self.coef_ = solution[0]
        return dict(coef_=solution[0], basis=B, knots=k) 

    def predict(self, Xpred, coef, knots, l=3):
        """Calculate the B-spline value for X given the parameters in coef.

        Paramters:
        ----------
        X : array      - Data of shape (n_samples, ) to evaluate the B-spline on.
        coef  : array  - Parameters of the B-spline.
        knots : array  - Knot sequence of the B-spline.
        l : int        - Order of the B-spline.

        Returns:
        --------
        s : array   - B-spline values on X for the given parameters.

        """
        if len(Xpred.shape) == 1:
            print("Prediction for 1-D Data".center(30, "-"))
            B = np.zeros((len(Xpred), len(coef)))
            for j in range(l, len(knots)-1):
                B[:,j-l] = self.basisfunction(Xpred, knots, j, l)
                
        elif Xpred.shape[1] == 2:
            print("Prediction for 2-D Data".center(30, "-"))
            n_samples = len(Xpred[:,0])
            B1pred, B2pred = np.zeros((n_samples, len(knots[0])-1-l[0])), np.zeros((n_samples, len(knots[1])-1-l[1]))
            B = np.zeros((n_samples, len(coef)))

            for j in range(l[0], len(knots[0])-1):
                B1pred[:,j-l[0]] = self.basisfunction(Xpred[:,0], knots[0], j, l[0]) 
            for j in range(l[1], len(knots[1])-1):
                B2pred[:,j-l[1]] = self.basisfunction(Xpred[:,1], knots[1], j, l[1]) 
            for i in range(n_samples):
                B[i,:] = np.kron(B2pred[i,:], B1pred[i,:])    
        else:
            print("Maximal dimension == 2!")
            return 
        s = B @ coef
        return s    


In [118]:
np.random.seed(2)
x = np.random.random(200)*100
xpred = np.linspace(0, max(x), 1000)
y = np.sin(x) + np.random.normal(size=x.shape)*0.2
ypredtrue = np.sin(xpred)
#px.scatter(x=x, y=y)

B = Bspline()
sol = B.fit(x,y,nr_splines=65, knot_type="q")
ypred = B.predict(xpred, sol["coef_"], sol["knots"])

---Prediction for 1-D Data----


In [119]:
mean_squared_error(y, sol["basis"]@sol["coef_"])

0.03096358246254299

In [120]:
mean_squared_error(ypred, ypredtrue)

0.03340634365498488